<a href="https://colab.research.google.com/github/Aulia2699/DeepLearningTask/blob/main/TaskWeek7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

NAMA  : AULIA RAHMA

NPM   : 2306319426

In [6]:
!pip install transformers
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# Load the model using the pipeline
pipe = pipeline("text-classification", model="mrm8488/deberta-v3-ft-financial-news-sentiment-analysis")

# Load the model and tokenizer directly
tokenizer = AutoTokenizer.from_pretrained("mrm8488/deberta-v3-ft-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/deberta-v3-ft-financial-news-sentiment-analysis")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [7]:
# Create a synthetic dataset
synthetic_data = [
    {"text": "The company reported excellent profits this quarter.", "label": "POSITIVE"},
    {"text": "The stock prices are expected to fall.", "label": "NEGATIVE"},
    {"text": "Sales have increased significantly.", "label": "POSITIVE"},
    {"text": "There are concerns about the new policies.", "label": "NEGATIVE"},
    {"text": "Investors are optimistic about the future.", "label": "POSITIVE"},
]

# Split into texts and labels
texts = [item["text"] for item in synthetic_data]
labels = [item["label"] for item in synthetic_data]


In [3]:
from sklearn.metrics import accuracy_score

# Run predictions
predictions = pipe(texts)
predicted_labels = [pred['label'] for pred in predictions]

# Calculate accuracy
accuracy = accuracy_score(labels, predicted_labels)
print(f'Initial Model Accuracy: {accuracy:.2f}')


Initial Model Accuracy: 0.00


In [4]:
from transformers import DebertaV2Model
import torch.nn as nn

class CustomDebertaModel(nn.Module):
    def __init__(self, model_name):
        super(CustomDebertaModel, self).__init__()
        self.deberta = DebertaV2Model.from_pretrained(model_name)
        self.classifier = nn.Linear(self.deberta.config.hidden_size, 2)  # Assuming binary classification

    def forward(self, input_ids, attention_mask):
        outputs = self.deberta(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.last_hidden_state[:, 0, :])  # Take the output of the first token
        return logits

# Initialize the custom model
custom_model = CustomDebertaModel("mrm8488/deberta-v3-ft-financial-news-sentiment-analysis")


In [5]:
# Prepare the inputs
inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")

# Get the model outputs
with torch.no_grad():
    custom_outputs = custom_model(inputs['input_ids'], inputs['attention_mask'])

# Apply softmax to get probabilities and predictions
predicted_labels_custom = torch.argmax(torch.softmax(custom_outputs, dim=1), dim=1).numpy()

# Map numerical labels back to original labels for comparison
predicted_labels_custom = ['POSITIVE' if pred == 1 else 'NEGATIVE' for pred in predicted_labels_custom]

# Calculate new accuracy
accuracy_custom = accuracy_score(labels, predicted_labels_custom)
print(f'Custom Model Accuracy: {accuracy_custom:.2f}')


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Custom Model Accuracy: 0.60
